In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
sys.path

['/Users/syeehyn/Desktop/fall2020/ieor4571/personalization/eggs',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python38.zip',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/lib-dynload',
 '',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages/IPython/extensions',
 '/Users/syeehyn/.ipython',
 '../']

In [4]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [5]:
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [10]:
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
datas = loading(spark, '../downloads')

In [7]:
datas

{'links': DataFrame[movieId: string, imdbId: string, tmdbId: string],
 'tags': DataFrame[userId: string, movieId: string, tag: string, timestamp: string],
 'genome-tags': DataFrame[tagId: string, tag: string],
 'ratings': DataFrame[userId: string, movieId: string, rating: string, timestamp: string],
 'genome-scores': DataFrame[movieId: string, tagId: string, relevance: string],
 'movies': DataFrame[movieId: string, title: string, genres: string]}

In [8]:
ratings = datas['ratings']


In [9]:
for key in datas.keys():
    print(key)
    datas[key].show(10)

links
+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
+-------+-------+------+
only showing top 10 rows

tags
+------+-------+--------------+----------+
|userId|movieId|           tag| timestamp|
+------+-------+--------------+----------+
|    14|    110|          epic|1443148538|
|    14|    110|      Medieval|1443148532|
|    14|    260|        sci-fi|1442169410|
|    14|    260|  space action|1442169421|
|    14|    318|  imdb top 250|1442615195|
|    14|    318|       justice|1442615192|
|    14|    480|     Dinosaurs|1443148563|
|    14|    593|psychothriller|1444014286|
|    14|   1682|    philosophy|1442615158|
|    14|   1682|  surveillance|1442615167|
+------+-------+--------------+----------+
only showin

In [10]:
movies = datas['movies']

In [11]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [12]:
movies.select('genres')

DataFrame[genres: string]

In [13]:
20000/1000

20.0

In [19]:
userid_filter = ratings.groupby('userId')\
        .agg(F.count(F.col('movieId'))\
                    .alias('count'))\
        .where(F.col('count') >= 5)
movieid_filter = ratings.groupby('movieId')\
        .agg(F.count(F.col('userId'))\
                    .alias('count'))\
        .where(F.col('count') >= 100)
subset = ratings.join(userid_filter,
                        ratings.userId == userid_filter.userId)\
                    .select(ratings.userId, ratings.movieId, ratings.rating)\
                    .join(movieid_filter,
                        ratings.movieId == movieid_filter.movieId)\
                    .select(ratings.userId, ratings.movieId, ratings.rating).persist()

In [20]:
subset.select('userId').distinct().count()

266971

In [21]:
subset.select('movieId').distinct().count()

10500

In [26]:
frac = .001
frac_incr = 2
min_user = 20000
min_movie = 1000
random_seed = 0
cnt_user = 0
cnt_movie = 0
while cnt_user < min_user or cnt_movie < min_movie:
    frac *= frac_incr
    sample = subset.sample(fraction=frac, seed = random_seed)
    userid_filter = sample.groupby('userId')\
            .agg(F.count(F.col('movieId'))\
                        .alias('count'))\
            .where(F.col('count') >= 5)
    movieid_filter = sample.groupby('movieId')\
            .agg(F.count(F.col('userId'))\
                        .alias('count'))\
        .where(F.col('count') >= 100)
    sample = sample.join(userid_filter,
                        sample.userId == userid_filter.userId)\
                    .select(sample.userId, sample.movieId, sample.rating)\
                    .join(movieid_filter,
                        sample.movieId == movieid_filter.movieId)\
                    .select(sample.userId, sample.movieId, sample.rating)
    cnt_user = sample.select('userId').distinct().count()
    cnt_movie = sample.select('movieId').distinct().count()
    print(f'''
            with frac = {frac},
            {cnt_user} users rated at least {5} movies,
            {cnt_movie} movies are rated by at least {100} users.
            ''')


            with frac = 0.002,
            75 users rated at least 5 movies,
            31 movies are rated by at least 100 users.
            

            with frac = 0.004,
            1685 users rated at least 5 movies,
            168 movies are rated by at least 100 users.
            

            with frac = 0.008,
            9520 users rated at least 5 movies,
            520 movies are rated by at least 100 users.
            

            with frac = 0.016,
            25472 users rated at least 5 movies,
            1106 movies are rated by at least 100 users.
            


In [28]:
sample.toPandas()

,userId,movieId,rating
0,101626,1,4.0
1,102241,1,4.0
2,104728,1,4.0
3,226331,1,2.0
4,226621,1,3.5
...,...,...,...
140097,79346,96610,4.5
140098,88187,96610,2.0
140099,101223,96610,3.0
140100,102122,96610,4.0
